# Data Analysis

In [1]:
import tensorflow as tf

In [2]:
from datasets import load_dataset
raw_dataset = load_dataset("squad")

c:\Users\acayt\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad (C:/Users/acayt/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 18.98it/s]


In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [18]:
type(raw_dataset)

datasets.dataset_dict.DatasetDict

In [4]:
raw_dataset.shape

{'train': (87599, 5), 'validation': (10570, 5)}

In [20]:
raw_dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [21]:
type(raw_dataset['train'])

datasets.arrow_dataset.Dataset

In [22]:
raw_dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [23]:
type(raw_dataset['train'][0])

dict

In [5]:
print("ID: ", raw_dataset['train'][0]['id'])
print("Title: ", raw_dataset['train'][0]['title'])
print("Context: ", raw_dataset['train'][0]['context'])
print("Question: ", raw_dataset['train'][0]['question'])
print("Answer: ", raw_dataset['train'][0]['answers'])

ID:  5733be284776f41900661182
Title:  University_of_Notre_Dame
Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


*Attributes `ID` and `Title` are not useful.*

In [28]:
type(raw_dataset['train'][0]['context']), type(raw_dataset['train'][0]['question']), type(raw_dataset['train'][0]['answers'])

(str, str, dict)

In [30]:
type(raw_dataset['train'][0]['answers']['text']), raw_dataset['train'][0]['answers']['text']

(list, ['Saint Bernadette Soubirous'])

In [31]:
type(raw_dataset['train'][0]['answers']['answer_start']), raw_dataset['train'][0]['answers']['answer_start']

(list, [515])

In [6]:
raw_dataset['train'].filter(lambda x: len(x['answers']['text']) != 1)

Loading cached processed dataset at C:/Users/acayt/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-6f50cc0ea6e240f9.arrow


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

*During training, there is only one possible answer. This is a double-check mechanism put in place.\
Evaluation set may contain multiple answers, but that's not of concern.*

# Preprocessing

In [7]:
from transformers import AutoTokenizer

In [8]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [33]:
?tokenizer

Signature:     
tokenizer(
    text: Union[str, List[str], List[List[str]]] = None,
    text_pair: Union[str, List[str], List[List[str]], NoneType] = None,
    text_target: Union[str, List[str], List[List[str]]] = None,
    text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = False,
    max_length: Optional[int] = None,
    stride: int = 0,
    is_split_into_words: bool = False,
    pad_to_multiple_of: Optional[int] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    return_token_type_ids: Optional[bool] = None,
    return_attention_mask: Optional[bool] = None,
    return_overflowing_tokens: bool = False,
    return_special_tokens_mask: bool = False,
    return_offsets_mapping: bool = False,
    return_length

In [9]:
tokenizer.is_fast

True

*A check to make sure that fast-tokenizer is implemented in the used model type (BERT).*

In [10]:
context = raw_dataset['train'][0]['context']
question = raw_dataset['train'][0]['question']

In [11]:
context, question

('Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')

In [46]:
temp = context + question
len(temp), temp

(766,
 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')

In [12]:
inputs = tokenizer(question, context)

In [41]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [39]:
len(inputs), inputs

(3,
 {'input_ids': [101, 1706, 2292, 1225, 1103, 6567, 2090, 9273, 2845, 1107, 8109, 1107, 10111, 20500, 1699, 136, 102, 22182, 1193, 117, 1103, 1278, 1144, 170, 2336, 1959, 119, 1335, 4184, 1103, 4304, 4334, 112, 188, 2284, 10945, 1110, 170, 5404, 5921, 1104, 1103, 6567, 2090, 119, 13301, 1107, 1524, 1104, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 171, 17506, 9538, 1110, 1103, 144, 10595, 2430, 117, 170, 14789, 1282, 1104, 8070, 1105, 9284, 119, 1135, 1110, 170, 16498, 1104, 1103, 176, 10595, 2430, 1120, 10111, 20500, 117, 1699, 1187, 1103, 6567, 2090, 25153, 1193, 1691, 1106, 2216, 17666, 6397, 3786, 1573, 25422, 13149, 1107, 8109, 119, 1335, 1103, 1322, 1104, 1103, 1514, 2797, 113, 1105, 1107, 170, 2904, 1413, 1115, 8200, 1194, 124, 11739, 110

In [15]:
inputs['input_ids']

[101,
 1706,
 2292,
 1225,
 1103,
 6567,
 2090,
 9273,
 2845,
 1107,
 8109,
 1107,
 10111,
 20500,
 1699,
 136,
 102,
 22182,
 1193,
 117,
 1103,
 1278,
 1144,
 170,
 2336,
 1959,
 119,
 1335,
 4184,
 1103,
 4304,
 4334,
 112,
 188,
 2284,
 10945,
 1110,
 170,
 5404,
 5921,
 1104,
 1103,
 6567,
 2090,
 119,
 13301,
 1107,
 1524,
 1104,
 1103,
 4304,
 4334,
 1105,
 4749,
 1122,
 117,
 1110,
 170,
 7335,
 5921,
 1104,
 4028,
 1114,
 1739,
 1146,
 14089,
 5591,
 1114,
 1103,
 7051,
 107,
 159,
 21462,
 1566,
 24930,
 2508,
 152,
 1306,
 3965,
 107,
 119,
 5893,
 1106,
 1103,
 4304,
 4334,
 1110,
 1103,
 19349,
 1104,
 1103,
 11373,
 4641,
 119,
 13301,
 1481,
 1103,
 171,
 17506,
 9538,
 1110,
 1103,
 144,
 10595,
 2430,
 117,
 170,
 14789,
 1282,
 1104,
 8070,
 1105,
 9284,
 119,
 1135,
 1110,
 170,
 16498,
 1104,
 1103,
 176,
 10595,
 2430,
 1120,
 10111,
 20500,
 117,
 1699,
 1187,
 1103,
 6567,
 2090,
 25153,
 1193,
 1691,
 1106,
 2216,
 17666,
 6397,
 3786,
 1573,
 25422,
 13149,
 11

In [42]:
type(inputs['input_ids']), len(inputs['input_ids'])

(list, 181)

In [43]:
type(inputs['token_type_ids']), len(inputs['token_type_ids'])

(list, 181)

In [44]:
type(inputs['attention_mask']), len(inputs['attention_mask'])

(list, 181)

In [14]:
tokenizer.decode(inputs['input_ids'])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

*The format seems like `[CLS]` question `[SEP]` context `[SEP]`.*

*Using Sliding window to deal with long features by splitting them.*

In [13]:
inputs = tokenizer(
    question,
    context,
    max_length=100, #length limit
    truncation="only_second", #to only split the 'Context'
    stride=50, #no of overlapping tokens between 2 successive windows
    return_overflowing_tokens=True, #to consider the overflowing tokens
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the B

*Context splitted into 4 windows. Answer appears in 3rd and 4th windows.*